In [0]:

# Ititialize Spark content
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('BigData')
sc=SparkContext(conf=conf)
print(sc)
print('Ready to Go!!!!')

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-4140719852775921>:5
      2 from pyspark import SparkContext, SparkConf
      4 conf = SparkConf().setAppName('BigData')
----> 5 sc=SparkContext(conf=conf)
      6 print(sc)
      7 print('Ready to Go!!!!')

File /databricks/spark/python/pyspark/context.py:202, in SparkContext.__init__(self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls, udf_profiler_cls, memory_profiler_cls)
    196 if gateway is not None and gateway.gateway_parameters.auth_token is None:
    197     raise ValueError(
    198         "You are trying to pass an insecure Py4j gateway to Spark. This"
    199         " is not allowed as it is a security risk."
    200     )
--> 202 SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
    203 try:
    204     self._do_init(
    205      

###  In case you are encounter with error "Cannot run multiple SparkContexts at once"
### Run the following command

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


In [0]:
# Get the Application Name:
print(sc.appName)


Databricks Shell


In [0]:
# Get Master URL
# This provides the master URL of the Spark cluster (local, yarn, etc.).
print(sc.master)


local[8]


In [0]:
# Get Other Details: You can also use sc.getConf() to retrieve the configuration of the current SparkContext:
print(sc.getConf().getAll())


[('spark.databricks.preemption.enabled', 'true'), ('spark.sql.hive.metastore.jars', '/databricks/databricks-hive/*'), ('spark.driver.tempDirectory', '/local_disk0/tmp'), ('spark.sql.warehouse.dir', 'dbfs:/user/hive/warehouse'), ('spark.driver.host', '10.172.224.178'), ('spark.databricks.managedCatalog.clientClassName', 'com.databricks.managedcatalog.ManagedCatalogClientImpl'), ('spark.databricks.credential.scope.fs.gs.auth.access.tokenProviderClassName', 'com.databricks.backend.daemon.driver.credentials.CredentialScopeGCPTokenProvider'), ('spark.hadoop.fs.fcfs-s3.impl.disable.cache', 'true'), ('spark.sql.streaming.checkpointFileManagerClass', 'com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager'), ('spark.databricks.service.dbutils.repl.backend', 'com.databricks.dbconnect.ReplDBUtils'), ('spark.hadoop.databricks.s3.verifyBucketExists.enabled', 'false'), ('spark.streaming.driver.writeAheadLog.allowBatching', 'true'), ('spark.databricks.clusterSource', 'UI'), ('spark.hadoo

In [0]:
# Reuse the Existing SparkContext: Avoid explicitly initializing SparkContext() and instead rely on getOrCreate() to use the existing one.
# Stop the Existing SparkContext: If you need to create a new context, stop the current one first:
sc.stop()
from pyspark import SparkContext
sc = SparkContext(appName="NewAppName")


In [0]:
# Use Databricks-Managed SparkContext: If working in Databricks, avoid manually creating SparkContext, as Databricks provides a pre-initialized SparkSession:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


In [0]:
# Initialize SparkSession
# Ensure you have a SparkSession object, which is the entry point for working with DataFrames.
from pyspark.sql import SparkSession

# Create or get the SparkSession
spark = SparkSession.builder \
    .appName("Read CSV File Example") \
    .getOrCreate()


In [0]:
# Read the CSV File
# You can use the read.csv() method to load the CSV file into a DataFrame.
# Read CSV file into DataFrame
df = spark.read.csv("dbfs:/FileStore/streaming/Countries1.csv")


In [0]:
# With Header and Inferred Schema:
# If the CSV file has a header and you want Spark to infer the schema:
df = spark.read.format("csv") \
    .option("header", "true") \
        .option("inferSchema", "true") \
            .load("dbfs:/FileStore/streaming/Countries1.csv")
        
    


In [0]:
#  How many partitions do we have?
# By default, the number of partitions is determined by the number of cores available
# in your local setup or cluster.
# If you are running it locally, it's often based on the number of CPU cores.
# Get the number of partitions
num_partitions = df.rdd.getNumPartitions()
print("Number of partitions:", num_partitions)
# Explanation
# df.rdd: Converts the DataFrame to its underlying RDD.
# getNumPartitions(): Returns the number of partitions of the RDD.

Number of partitions: 1


In [0]:
print("Type of num_partitions:", type(num_partitions))

Type of num_partitions: <class 'int'>


In [0]:
rdd = df.rdd
print("Type of rdd:", type(rdd))

Type of rdd: <class 'pyspark.rdd.RDD'>


In [0]:
rdd.glom().map(len).collect()

# glom(): Transforms each partition of the RDD into a list. Instead of working with individual elements, you now have a list of elements for each partition.

# map(len): Applies the len function to each partition (which is now a list) to get the count of elements in that partition.

# collect(): Collects the result back to the driver as a list, giving the count of elements in each partition.
     

Out[24]: [6]

In [0]:
# To check the contents of the RDD
print(rdd.collect())

[Row(Country='India', Citizens=10), Row(Country='USA', Citizens=5), Row(Country='China', Citizens=10), Row(Country='India', Citizens=10), Row(Country='Canada', Citizens=40), Row(Country='Brazil', Citizens=10)]


In [0]:

rdd =rdd.repartition(10)

# Can increase or decrease the level of parallelism in this RDD.
# Internally, this uses a shuffle to redistribute data.
# If you are decreasing the number of partitions in this RDD, consider using coalesce,
#  which can avoid performing a shuffle.

In [0]:

rdd.glom().map(len).collect()

Out[37]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 6]

In [0]:
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

rdd.persist()
# 2 cores and 10 partitions, 5 partitions in each core

8
10
Out[38]: MapPartitionsRDD[73] at coalesce at NativeMethodAccessorImpl.java:0

In [0]:
# The output of your rdd.glom().map(len).collect() indicates that your existing RDD is unevenly distributed across partitions, with 9 partitions having no data and 1 partition containing all the elements (6). This imbalance can affect performance in distributed processing. To address this, you can repartition the RDD to distribute the data more evenly.

# Here’s how you can parallelize and balance your existing RDD:
# Repartition the RDD into a desired number of partitions (e.g., 10)
balanced_rdd = rdd.repartition(10)

# Verify the new partition sizes
print(balanced_rdd.glom().map(len).collect())


[0, 0, 0, 0, 0, 0, 0, 0, 0, 6]


In [0]:
# Inspect the data distribution across partitions
print(rdd.glom().collect())


[[], [], [], [], [], [], [], [], [], [Row(Country='India', Citizens=10), Row(Country='USA', Citizens=5), Row(Country='China', Citizens=10), Row(Country='India', Citizens=10), Row(Country='Canada', Citizens=40), Row(Country='Brazil', Citizens=10)]]


In [0]:
# You can explicitly recreate a balanced RDD using sparkContext.parallelize(). Here’s how to assign and parallelize data properly:

# Collect the RDD to a Python list
data_list = rdd.collect()

# Re-parallelize the data with the desired number of partitions
balanced_rdd = spark.sparkContext.parallelize(data_list, 10)

# Verify the new partition distribution
print(balanced_rdd.glom().map(len).collect())


[0, 1, 0, 1, 1, 0, 1, 0, 1, 1]


In [0]:
# Using Repartition with Larger Datasets
# If your dataset is large enough, repartition() should naturally distribute it. However, for small datasets, repartitioning may still leave some partitions empty. Here's how to ensure shuffling:

# Force shuffle and redistribute the data
balanced_rdd = rdd.repartition(4)

# Force an action to materialize the redistribution
balanced_rdd.count()

# Check the distribution again
print(balanced_rdd.glom().map(len).collect())


[0, 0, 0, 6]


In [0]:
print("Original partition data:", rdd.glom().collect())
print("Balanced partition data:", balanced_rdd.glom().collect())


Original partition data: [[], [], [], [], [], [], [], [], [], [Row(Country='India', Citizens=10), Row(Country='USA', Citizens=5), Row(Country='China', Citizens=10), Row(Country='India', Citizens=10), Row(Country='Canada', Citizens=40), Row(Country='Brazil', Citizens=10)]]
Balanced partition data: [[], [Row(Country='India', Citizens=10)], [], [Row(Country='USA', Citizens=5)], [Row(Country='China', Citizens=10)], [], [Row(Country='India', Citizens=10)], [], [Row(Country='Canada', Citizens=40)], [Row(Country='Brazil', Citizens=10)]]


In [0]:

# Group by the first element (name)
grouped_rdd = balanced_rdd.groupBy(lambda x: x[0])

# Process each group
result = grouped_rdd.mapValues(lambda x: list(x)).collect()

# Print the grouped data
for key, value in result:
    print(f"{key}: {value}")

India: [Row(Country='India', Citizens=10), Row(Country='India', Citizens=10)]
USA: [Row(Country='USA', Citizens=5)]
Brazil: [Row(Country='Brazil', Citizens=10)]
Canada: [Row(Country='Canada', Citizens=40)]
China: [Row(Country='China', Citizens=10)]


In [0]:
# Group by the first element (name)
grouped_rdd = rdd.groupBy(lambda x: x[0])

# Process each group
result = grouped_rdd.mapValues(lambda x: list(x)).collect()

# Print the grouped data
for key, value in result:
    print(f"{key}: {value}")

In [0]:
from pyspark.sql.functions import *

emp_df=rdd.filter(col("Country")).select().groupby

In [0]:
grouped_rdd = rdd.groupByKey().mapValues(sum)

# Collect and print the grouped data
print("Grouped Data:", grouped_rdd.collect())


Grouped Data: [('India', 20), ('USA', 5), ('Brazil', 10), ('Canada', 40), ('China', 10)]


In [0]:
# With Header and Inferred Schema:
# If the CSV file has a header and you want Spark to infer the schema:
df = spark.read.format("csv") \
    .option("header", "true") \
            .load("dbfs:/FileStore/streaming/Countries1.csv")

In [0]:
print(df.rdd.getNumPartitions())

1


In [0]:
partition_df = df.repartition(6)

In [0]:
print(partition_df)

DataFrame[Country: string, Citizens: string]


In [0]:
print(partition_df.rdd.getNumPartitions())

6


In [0]:
from pyspark.sql.functions import col

partition_df=partition_df.filter(col("Country")=="India")
partition_df.collect()

Out[56]: [Row(Country='India', Citizens='10'), Row(Country='India', Citizens='10')]

In [0]:

partition_df=partition_df.filter(col("Country")=="India")\
    .select("Country","Citizens")
partition_df.collect()

Out[57]: [Row(Country='India', Citizens='10'), Row(Country='India', Citizens='10')]

In [0]:
partition_df=partition_df.filter(col("Country")=="India")\
    .select("Country","Citizens")\
        .groupby("Country").count()
partition_df.collect()

Out[58]: [Row(Country='India', count=2)]